In [ ]:
using DrWatson

In [ ]:
@quickactivate "RxInferThesisExperiments"

In [ ]:
using RxInferThesisExperiments, Plots, PGFPlotsX, LaTeXStrings, Printf
using LinearAlgebra, StableRNGs, Random, BenchmarkTools, ColorSchemes, Dates, DataFrames

import RxInfer, ReactiveMP, ForneyLab, Turing

In [ ]:
pgfplotsx()

# gr()

In [ ]:
const outfolder = plotsdir("lds")

In [ ]:
mkpath(outfolder);

In [ ]:
function analyze_benchmarks(filterfunction, bfolder)
    benchmarks = prepare_benchmarks_table(bfolder);

    # Select only a portion of benchmarks for plotting
    filtered = filter(filterfunction, benchmarks)

    sorted = sort(filtered, [ :T ])

    # RxInfer includes the model creation time in it
    inference = getindex.(sorted.inference, 1) .- getindex.(sorted.inference, 3)
    creation  = getindex.(sorted.creation, 1) .- getindex.(sorted.creation, 3)

    min_timing_range = min(minimum(inference), minimum(creation))
    max_timing_range = max(maximum(inference), maximum(creation))

    return sorted, (inference, creation), (min_timing_range, max_timing_range)
end

In [ ]:
target_seed = 42
target_d = 4

In [ ]:
rxifb, (rxi_inference, rxi_creation), (rxi_min_tr, rxi_max_tr) = analyze_benchmarks(datadir("lds", "rxinfer")) do r
    return r["d"] == target_d && r["seed"] == target_seed
end

In [ ]:
flfb, (fl_inference, fl_creation), (fl_min_tr, fl_max_tr) = analyze_benchmarks(datadir("lds", "forneylab")) do r
    return r["d"] == target_d && r["seed"] == target_seed
end

In [ ]:
tgfb100, (tg_inference100, tg_creation100), (tg_min_tr100, tg_max_tr100) = analyze_benchmarks(datadir("lds", "turing", "nuts")) do r
    return r["d"] == target_d && r["seed"] == target_seed && r["nsamples"] == 100
end

tgfb200, (tg_inference200, tg_creation200), (tg_min_tr200, tg_max_tr200) = analyze_benchmarks(datadir("lds", "turing", "nuts")) do r
    return r["d"] == target_d && r["seed"] == target_seed && r["nsamples"] == 200
end

In [ ]:
min_timing_range = min(rxi_min_tr, fl_min_tr, tg_min_tr100, tg_min_tr200)
max_timing_range = max(rxi_max_tr, fl_max_tr, tg_max_tr100, tg_max_tr200)

timing_range = exp.(range(log(min_timing_range), log(max_timing_range); length = 10))
sizes_range = sort(collect(union(rxifb.T, flfb.T, tgfb.T)))

pfontsettings = (
    titlefontsize=18,
    guidefontsize=16,
    tickfontsize=14,
    legendfontsize=14,
    legend_font_halign = :left
)

xticks = (
    [ 10, 100, 1000, 10_000, 100_000 ],
    [ L"10^1", L"10^2", L"10^3", L"10^4", L"10^5" ]
)

yticks = (
    [ 0.1, 1.0, 10.0, 100, 1000, 10000, 100000 ], 
    [ L"10^{-1}", L"10^{0}", L"10^{1}", L"10^{2}", L"10^{3}", L"10^{4}", L"10^5" ]
)

p = plot(
    size = (800, 400),
    yscale = :log10, xscale = :log10, xticks = xticks, yticks = yticks, 
    ylims = (minimum(yticks[1]), maximum(yticks[1])),
    ylabel = "Time (in ms, log-scale)", xlabel = "Number of observation (log-scale)",
    legend = :outerright;
    pfontsettings...
)

mshapes = [:utriangle, :dtriangle, :rtriangle, :ltriangle, :diamond, :circle, :rect, :utriangle ]
styles = [ :solid, :dash, :dot, :dashdot ]

p = plot!(p, rxifb.T, to_ms.(rxi_inference), label = "Reactive MP", marker = mshapes[1], style = styles[1])
p = plot!(p, flfb.T, to_ms.(fl_inference), label = "Scheduled MP (inference)", marker = mshapes[2], style = styles[2])
p = plot!(p, flfb.T, to_ms.(fl_creation), label = "Scheduled MP (compilation)", marker = mshapes[3], style = styles[2])
p = plot!(p, tgfb.T, to_ms.(tg_inference100), label = "NUTS (100)", marker = mshapes[4], style = styles[3])
p = plot!(p, tgfb.T, to_ms.(tg_inference200), label = "NUTS (200)", marker = mshapes[5], style = styles[3])

savefig(joinpath(outfolder, "02-benchmark_comparison.tex"))

display("image/png", p)